In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

from tqdm import tqdm
from sklearn.metrics import f1_score
import numpy as np
import random

# 시드 및 모델의 기본적인 파라미터 고정

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

batch_size = 32
num_epochs = 100
learning_rate = 0.0001

# GPU 확인 및 device에 사용할 gpu 저장

In [3]:
print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))
device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.7.1
GPU 사용 가능 여부: True


# 데이터셋 만들기

In [4]:
class SeperateDataset(Dataset): # 각 목표별 데이터셋 만들기
    def __init__(self, df, transform, target):
        self.df = df
        self.img_paths = self.df['path'].tolist()
        self.labels = self.df[target].tolist()
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(self.labels[index])

    def __len__(self):
        return len(self.img_paths)
    
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

# transform 만들기
### 아래 부분은 3x384x384 할지 3x224x224할지 조건에 따라 선택해서 사용하시면 됩니다.
### 각각 resize 후에 80퍼센트만 남도록 crop을 하였습니다.

In [5]:
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize((280,280)),
#         transforms.CenterCrop(224),
#         transforms.RandomRotation(5),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize((280,280)),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }



# 아래 영역을 마우스로 지정하고, 컨트롤키 + '/' 눌러서 주석 제거
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((480,480)),
        transforms.CenterCrop(384),
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((480,480)),
        transforms.CenterCrop(384),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 학습할 데이터 불러오기
### 이 아래 부분은 실험 조건에 따라서 주석처리하면서 돌리면 됩니다.

In [6]:
# 5:1:1
#df = pd.read_csv("/opt/ml/code/total.csv")

# 1:1:1
df = pd.read_csv("/opt/ml/code/total_111.csv", index_col=None)
list_columns = list(df.columns)

In [7]:
test_dir = '/opt/ml/input/data/eval/'

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

### 만들 타겟을 아래에 적어주세요 (gender/age/maskOX/maskGB/class)

In [8]:
target ="class"
list_columns.remove(target)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[list_columns], df[target], test_size=0.2, stratify=df[target], random_state=42)

In [10]:
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

In [11]:
train_data = SeperateDataset(train_df, transform=data_transforms['train'], target=target)
val_data = SeperateDataset(test_df, transform=data_transforms['test'], target=target)
test_data = TestDataset(image_paths, transform=data_transforms['test'])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [12]:
import timeit

def train(num_epochs, model, data_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    total_start_time = timeit.default_timer()
    best_loss = 9999999
    best_test_accuracy = 0
    for epoch in tqdm(range(num_epochs)):
        epoch_f1 = 0
        running_acc = 0
        print('Epoch start..')
        epoch_start_time = timeit.default_timer()
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()         
            loss.backward()
            optimizer.step()

            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax).float().mean()
            
            f1 = f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
            epoch_f1 += f1
            running_acc += accuracy
            if (i+1) % 3 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, i+1, len(data_loader), loss.item(), accuracy.item() * 100, f1))
        print("------------Epoch Finish------------")
        print('Epoch [{}/{}], Accuracy: {:.2f}%, F1_Score: {:.2f}'.format(
                    epoch+1, num_epochs, running_acc.item()/(i+1) * 100,epoch_f1/(i+1)))
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)
        epoch_end_time = timeit.default_timer()
        print("Epoch end..")
        print(f"epoch time : {epoch_end_time-epoch_start_time}")
        epoch_acc = running_acc / (i+1)
        
        if best_test_accuracy < epoch_acc:
            best_test_accuracy = epoch_acc
            save_model(model, saved_dir)
            early_stop_point = 0
        else:
            early_stop_point += 1
        if early_stop_point == 3:
            print('early_stopped')
            break
    print('End training..')
    total_end_time = timeit.default_timer()
    print(f"total time : {total_end_time-total_start_time}")

In [13]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch) )
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        total_loss = 0
        cnt = 0
        epoch_f1 = 0
        for i, (imgs, labels) in enumerate(data_loader):
            imgs, labels = imgs.to(device), labels.to(device)
            ## 코드 시작 ##
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            ## 코드 종료 ##
            total += imgs.size(0)
            _, argmax = torch.max(outputs, 1)
            correct += (labels == argmax).sum().item()
            total_loss += loss
            cnt += 1
            epoch_f1 += f1_score(labels.cpu().numpy(), argmax.cpu().numpy(), average='macro')
        avrg_loss = total_loss / cnt
        avrg_f1 = epoch_f1 / cnt
        print('Validation #{}  Accuracy: {:.2f}% F1_Score: {:.2f} Average Loss: {:.4f}'.format(epoch, correct / total * 100,avrg_f1 ,avrg_loss))
    model.train()
    return avrg_loss

In [14]:
def save_model(model, saved_dir, file_name='best_model.pt'):
    os.makedirs(saved_dir, exist_ok=True)
    check_point = {
        'net': model.state_dict()
    }
    output_path = os.path.join(saved_dir, file_name)
    torch.save(check_point,output_path)

In [15]:
# pip install timm

In [16]:
import torchvision

resnet18 = torchvision.models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /opt/ml/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [20]:
import math

OUTPUT_CLASS_NUM = 18
resnet18.fc = torch.nn.Linear(in_features = 512, out_features = OUTPUT_CLASS_NUM, bias = True)

torch.nn.init.xavier_uniform(resnet18.fc.weight)
stdv = 1. / math.sqrt(resnet18.fc.weight.size(1))
resnet18.fc.bias.data.uniform_(-stdv, stdv)

model = resnet18

<ipython-input-20-8b4ed5ea7ccb>:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(resnet18.fc.weight)


In [22]:
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

saved_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model'
val_every = 1

In [24]:
train(num_epochs, model, train_loader, criterion, optimizer, saved_dir, val_every, device)

  0%|          | 0/100 [00:00<?, ?it/s]

Start training..
Epoch start..
Epoch [1/100], Step [3/202], Loss: 3.2477, Accuracy: 9.38%, F1_Score: 0.02
Epoch [1/100], Step [6/202], Loss: 2.4340, Accuracy: 25.00%, F1_Score: 0.18
Epoch [1/100], Step [9/202], Loss: 2.0688, Accuracy: 31.25%, F1_Score: 0.16
Epoch [1/100], Step [12/202], Loss: 1.3803, Accuracy: 53.12%, F1_Score: 0.34
Epoch [1/100], Step [15/202], Loss: 1.4256, Accuracy: 59.38%, F1_Score: 0.44
Epoch [1/100], Step [18/202], Loss: 0.7208, Accuracy: 87.50%, F1_Score: 0.72
Epoch [1/100], Step [21/202], Loss: 0.8637, Accuracy: 75.00%, F1_Score: 0.63
Epoch [1/100], Step [24/202], Loss: 1.1246, Accuracy: 65.62%, F1_Score: 0.44
Epoch [1/100], Step [27/202], Loss: 0.9759, Accuracy: 75.00%, F1_Score: 0.65
Epoch [1/100], Step [30/202], Loss: 0.8383, Accuracy: 81.25%, F1_Score: 0.62
Epoch [1/100], Step [33/202], Loss: 0.4063, Accuracy: 93.75%, F1_Score: 0.94
Epoch [1/100], Step [36/202], Loss: 0.7204, Accuracy: 81.25%, F1_Score: 0.65
Epoch [1/100], Step [39/202], Loss: 0.6810, Accur

  1%|          | 1/100 [01:48<2:58:32, 108.20s/it]

Epoch start..
Epoch [2/100], Step [3/202], Loss: 0.3120, Accuracy: 93.75%, F1_Score: 0.94
Epoch [2/100], Step [6/202], Loss: 0.1211, Accuracy: 96.88%, F1_Score: 0.90
Epoch [2/100], Step [9/202], Loss: 0.1997, Accuracy: 96.88%, F1_Score: 0.98
Epoch [2/100], Step [12/202], Loss: 0.3852, Accuracy: 87.50%, F1_Score: 0.84
Epoch [2/100], Step [15/202], Loss: 0.0831, Accuracy: 96.88%, F1_Score: 0.96
Epoch [2/100], Step [18/202], Loss: 0.3027, Accuracy: 84.38%, F1_Score: 0.62
Epoch [2/100], Step [21/202], Loss: 0.1751, Accuracy: 96.88%, F1_Score: 0.90
Epoch [2/100], Step [24/202], Loss: 0.1198, Accuracy: 100.00%, F1_Score: 1.00
Epoch [2/100], Step [27/202], Loss: 0.4484, Accuracy: 81.25%, F1_Score: 0.75
Epoch [2/100], Step [30/202], Loss: 0.2703, Accuracy: 90.62%, F1_Score: 0.83
Epoch [2/100], Step [33/202], Loss: 0.3557, Accuracy: 87.50%, F1_Score: 0.85
Epoch [2/100], Step [36/202], Loss: 0.1744, Accuracy: 93.75%, F1_Score: 0.86
Epoch [2/100], Step [39/202], Loss: 0.1857, Accuracy: 90.62%, F1

  2%|▏         | 2/100 [03:34<2:55:38, 107.53s/it]

...

Validation #18  Accuracy: 93.69% F1_Score: 0.88 Average Loss: 0.2372
Epoch end..
epoch time : 104.85874682600843
Epoch start..
Epoch [19/100], Step [3/202], Loss: 0.0614, Accuracy: 96.88%, F1_Score: 0.90
Epoch [19/100], Step [6/202], Loss: 0.0089, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [9/202], Loss: 0.0013, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [12/202], Loss: 0.0077, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [15/202], Loss: 0.0287, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [18/202], Loss: 0.0072, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [21/202], Loss: 0.0046, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [24/202], Loss: 0.0128, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [27/202], Loss: 0.0076, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [30/202], Loss: 0.0051, Accuracy: 100.00%, F1_Score: 1.00
Epoch [19/100], Step [33/202], Loss: 0.0032, Accuracy: 100.00%, F1_Score: 1.00
Epoch [1

 19%|█▉        | 19/100 [33:30<2:22:18, 105.41s/it]

Validation #19  Accuracy: 91.06% F1_Score: 0.86 Average Loss: 0.3684
Epoch end..
epoch time : 105.25415464799153
Epoch start..
Epoch [20/100], Step [3/202], Loss: 0.0444, Accuracy: 96.88%, F1_Score: 0.98
Epoch [20/100], Step [6/202], Loss: 0.0176, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [9/202], Loss: 0.0954, Accuracy: 96.88%, F1_Score: 0.96
Epoch [20/100], Step [12/202], Loss: 0.0712, Accuracy: 96.88%, F1_Score: 0.97
Epoch [20/100], Step [15/202], Loss: 0.0153, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [18/202], Loss: 0.1167, Accuracy: 90.62%, F1_Score: 0.92
Epoch [20/100], Step [21/202], Loss: 0.0239, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [24/202], Loss: 0.0058, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [27/202], Loss: 0.0074, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [30/202], Loss: 0.0312, Accuracy: 100.00%, F1_Score: 1.00
Epoch [20/100], Step [33/202], Loss: 0.0427, Accuracy: 96.88%, F1_Score: 0.97
Epoch [20/10

 19%|█▉        | 19/100 [35:15<2:30:18, 111.34s/it]

Validation #20  Accuracy: 91.75% F1_Score: 0.87 Average Loss: 0.3144
Epoch end..
epoch time : 105.12055225804215
early_stopped
End training..
total time : 2115.370427688991


In [25]:
model_path = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/model/best_model.pt'
checkpoint = torch.load(model_path,map_location=device)
state_dict = checkpoint['net']
model.load_state_dict(state_dict)

<All keys matched successfully>

In [26]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))


# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in test_loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submit_dir = '/opt/ml/level1-image-classification-level1-recsys-16/junghkim/submit'
submission.to_csv(os.path.join(submit_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!
